In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import json
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from icecream import ic
from import_casa import casa, cano
from casa.annot import AspectEnum

In [3]:
TARGET_DIR = "20210412"
result_dir = casa.get_data_path() / f"annot_data/annotated_data_bkup/{TARGET_DIR}"
json_paths = [x for x in result_dir.iterdir() if x.suffix==".json"]

In [4]:
with json_paths[1].open("r", encoding="UTF-8") as fin:
    annots = json.load(fin)

In [5]:
list(annots[0].keys())

['completions', 'data', 'id']

## Main conversion loop

In [30]:
seq_pairs_list = []
for json_path in tqdm(json_paths):
    with json_path.open("r", encoding="UTF-8") as fin:
        annots = json.load(fin)
    for annot_i, annot_x in enumerate(annots): 
        try:
            aspects = cano.process_thread_annotations(annot_x)
            seq_pairs = cano.make_sequence_from_aspects(aspects, annot_x["data"]["html"])
            seq_pairs_list.extend(seq_pairs)
        except Exception:
            print(annot_i)
            raise Exception()

 10%|████████▎                                                                          | 1/10 [00:00<00:01,  7.71it/s]

different text spans:  方便又迅速！Good!


 20%|████████████████▌                                                                  | 2/10 [00:00<00:02,  3.28it/s]

約翰是哪裡?
不確定是什麼
多tag
可以算涵蓋嗎...?
地區+時段
metaphorical?


 30%|████████████████████████▉                                                          | 3/10 [00:01<00:02,  2.99it/s]

不確定指哪裡
安平區
台北市區
499之亂後
NP


 40%|█████████████████████████████████▏                                                 | 4/10 [00:01<00:01,  3.06it/s]

日本國際漫遊
成功嶺
海景
different text spans:  前18個月319/一個月，後六個月一個月399
different text spans:  中華電信5G有在建設？？出門連到5G的機會還是不到1/10
different text spans:  連玩coin master都會斷線的電信我真的不知道要說什麼
different text spans:  每家都有表現差的狀況，就是看環境適合誰囉，我生活圈台哥就是順
different text spans:  用起來是比其他家都還快
different text spans:  不過遠傳這次拿雙冠真的有讓人刮目相看
sarcasm


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  4.17it/s]

test
test


In [31]:
len(seq_pairs_list)

2242

## Saving data

In [32]:
with open(result_dir / f"seq_pairs_{TARGET_DIR}.pkl", "wb") as fout:
    pickle.dump(seq_pairs_list, fout)

In [33]:
cano.print_seq_pair(seq_pairs_list[0])

台(B-C) 星(I-C) 的(I-C) 態(I-C) 度(I-C) 就(I-C) 是(I-C) 在(I-C) 等(I-C) 宿(I-C) 主(I-C) 台(I-C) 哥(I-C) ，(I-C) 逸(I-C) 以(I-C) 待(I-C) 勞(I-C) ，(I-C) 準(I-C) 備(I-C) 寄(I-C) 生(I-C) 。(I-C)


In [34]:
ftext = open(result_dir / f"seq_pairs_{TARGET_DIR}.text.txt", "w", encoding="UTF-8")
ftags = open(result_dir / f"seq_pairs_{TARGET_DIR}.tags.txt", "w", encoding="UTF-8")
for text, tags in seq_pairs_list:
    ftext.write(text + "\n")
    ftags.write(" ".join(tags) + "\n")
ftext.close()
ftags.close()

## Sandbox

In [7]:
def find_thread_idx(annots, thread_idx):
    iter_annot = filter(lambda x: x["data"]["thread_idx"]==thread_idx, annots)
    return list(iter_annot)[0]

In [8]:
annots = []
for json_path in tqdm(json_paths):
    with json_path.open("r", encoding="UTF-8") as fin:
        obj = json.load(fin)
    annots.extend(obj)        


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 42.29it/s]


In [9]:
annot_x = find_thread_idx(annots, 220)

In [10]:
aspects, dbg = cano.process_thread_annotations(annot_x, True)
seq_pairs = cano.make_sequence_from_aspects(aspects, annot_x["data"]["html"])

c:\python38\lib\site-packages\html5lib\_ihatexml.py:258: DataLossWarning: Coercing non-XML name: '
  warnings.warn("Coercing non-XML name: %s" % name, DataLossWarning)


In [11]:
cano.print_seq_pair(seq_pairs[0])

先(B-O) 辦(B-O) 試(B-O) 用(B-O) ，(B-O) 中(B-E) 華(I-E) 4(B-A) 8(I-A) 8(I-A) 優(B-V) 惠(I-V) 不(I-V) 錯(I-V)
